# Preprocessing Data with TF

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= '2.0'
from tensorflow import keras

import numpy as np
import pandas as pd
import os
import time

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [15]:
tf.random.set_seed(42)

### Using California Dataset

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, 
    housing.target.reshape(-1, 1), 
    random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [4]:
scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

Split large dataset to multiple files:

In [5]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [6]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

In [7]:
train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [8]:
pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


----
## Creating a Pipeline

In [9]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [10]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [11]:
for line in dataset.take(5):
    print(line.numpy())

b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782'
b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'
b'3.3456,37.0,4.514084507042254,0.9084507042253521,458.0,3.2253521126760565,36.67,-121.7,2.526'
b'3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442'


Shuffle is working, but our result is byte strings. Convertion to Tensors is needed

In [12]:
n_inputs = X_train.shape[-1]

In [13]:
@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [14]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [17]:
train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(1):
    print("X =", X_batch)
    print("y =", y_batch)

X = tf.Tensor(
[[ 1.1832466  -0.2867314   0.256955   -0.0914653   0.6741611   0.05366582
  -0.7432092   0.71184903]
 [-0.44522637  1.8491895  -0.32066625 -0.14044929 -0.10611927 -0.06691425
  -0.691678    0.7318402 ]
 [ 0.3091969   0.5043504   0.20859428 -0.2770272   0.6084533   0.27369827
  -0.84627515  0.7818199 ]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[3.151]
 [2.226]
 [2.141]], shape=(3, 1), dtype=float32)


In [18]:
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [19]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [20]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [21]:
batch_size = 32
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=valid_set)

Epoch 1/10
362/362 [==============================] - 1s 2ms/step - loss: 1.6459 - val_loss: 0.8064
Epoch 2/10
362/362 [==============================] - 0s 1ms/step - loss: 0.6938 - val_loss: 0.6512
Epoch 3/10
362/362 [==============================] - 0s 1ms/step - loss: 0.6137 - val_loss: 0.6055
Epoch 4/10
362/362 [==============================] - 0s 984us/step - loss: 0.6218 - val_loss: 0.5653
Epoch 5/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5882 - val_loss: 0.5532
Epoch 6/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5346 - val_loss: 0.5153
Epoch 7/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5226 - val_loss: 0.5109
Epoch 8/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5075 - val_loss: 0.4800
Epoch 9/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5038 - val_loss: 0.4664
Epoch 10/10
362/362 [==============================] - 0s 1ms/step - loss: 0.4993 - val_loss: 0.47

In [22]:
model.evaluate(test_set, steps=len(X_test) // batch_size)

161/161 [==============================] - 0s 787us/step - loss: 0.4757


0.4756550192832947

----
## Using Pretrained Models

In [24]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [27]:
import tensorflow_hub as hub

In [30]:
!pip install --quiet tensorflow_text

ERROR: Could not find a version that satisfies the requirement tensorflow_text (from versions: none)
ERROR: No matching distribution found for tensorflow_text


In [25]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/wiki40b-lm-ru/1",
                           output_shape=[50], input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

ModuleNotFoundError: No module named 'tensorflow_hub'